In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from tqdm.auto import tqdm


from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from sklearn.preprocessing import StandardScaler

/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def check_for_multimodality(df, features):
    """
    Использует кластеризацию для поиска признаков с мультимодальным распределением,
    что может указывать на проблемы с масштабом или нестационарностью.
    """
    print("\n--- 5. Проверка на мультимодальность (Кластеризация) ---")
    multimodal_features = {}
    
    for feature in tqdm(features, desc="Кластеризация признаков"):
        # Извлекаем данные признака, убираем NaN и бесконечности
        feature_data = df[[feature]].replace([np.inf, -np.inf], np.nan).dropna()
        
        # Кластеризация имеет смысл только если у нас достаточно разнообразных данных
        if feature_data.empty or feature_data[feature].nunique() < 20:
            continue
            
        # Масштабируем данные, чтобы оценка была объективной
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(feature_data)
        
        try:
            # Пытаемся разделить на 2 кластера
            kmeans = KMeans(n_clusters=2, n_init='auto', random_state=42)
            kmeans.fit(X_scaled)
            
            # Считаем, насколько хорошим получилось разделение
            score = calinski_harabasz_score(X_scaled, kmeans.labels_)
            
            # --- ЭВРИСТИЧЕСКИЙ ПОРОГ ---
            # Если балл очень высокий, это значит, что данные легко разделились на 2 далекие группы
            # Этот порог можно и нужно настраивать. 5000 - это уже довольно значимое разделение.
            if score > 5000:
                multimodal_features[feature] = score
        except Exception:
            # Пропускаем, если кластеризация не удалась по какой-то причине
            continue
            
    return multimodal_features

In [3]:
# ==============================================================================
# КОНФИГУРАЦИЯ "КРАСНЫХ ФЛАГОВ"
# ==============================================================================
# Вы можете настраивать эти пороги, чтобы сделать диагностику более или менее строгой.
CFG = {
    "leakage_corr_threshold": 0.5,    # Корреляция с целью > 50% -> Подозрение на утечку
    "stationarity_pvalue_threshold": 0.05,  # p-value > 0.05 -> Подозрение на нестационарность
    "redundancy_corr_threshold": 0.95,  # Корреляция между признаками > 95% -> Избыточность
    "low_variance_threshold": 1e-4      # Дисперсия < 0.0001 -> Подозрение на бесполезность
}

# ==============================================================================
# ДИАГНОСТИЧЕСКИЕ ФУНКЦИИ
# ==============================================================================

def check_for_data_leakage(df, tbm_cols, features):
    """Проверяет признаки на аномально высокую корреляцию с целевыми переменными."""
    print("\n--- 1. Проверка на потенциальную утечку данных ---")
    leaky_features = {}
    for tbm_col in tbm_cols:
        correlations = df[features + [tbm_col]].corr()[tbm_col].abs().sort_values(ascending=False)
        # Исключаем саму себя (корреляция 1.0) и смотрим на остальные
        suspicious = correlations[1:][correlations > CFG["leakage_corr_threshold"]]
        if not suspicious.empty:
            leaky_features[tbm_col] = suspicious.to_dict()
    
    return leaky_features

# ЗАМЕНИТЕ СТАРУЮ ФУНКЦИЮ НА ЭТУ

def check_for_stationarity(df, features):
    """Проверяет каждый признак на стационарность с помощью теста Дики-Фуллера."""
    print("\n--- 2. Проверка на нестационарность (ADF-тест) ---")
    non_stationary_features = {}
    sample_tickers = df['Ticker'].unique()[:5] 
    
    for feature in tqdm(features, desc="ADF-тест признаков"):
        p_values = []
        for ticker in sample_tickers:
            series = df[df['Ticker'] == ticker][feature].dropna()
            
            # --- НОВЫЙ БЛОК ПРОВЕРКИ ---
            # Проверяем, достаточно ли данных И не является ли ряд константой
            if len(series) > 100 and len(series.unique()) > 1:
                try:
                    adf_result = adfuller(series)
                    p_values.append(adf_result[1]) # p-value
                except Exception as e:
                    # Ловим другие редкие математические ошибки, если они возникнут
                    print(f"\nПредупреждение: Не удалось провести ADF-тест для '{feature}' тикера '{ticker}'. Ошибка: {e}")
            # ---------------------------
        
        if p_values and np.mean(p_values) > CFG["stationarity_pvalue_threshold"]:
            non_stationary_features[feature] = np.mean(p_values)
            
    return non_stationary_features

def check_for_redundancy(df, features):
    """Проверяет признаки на высокую взаимную корреляцию (мультиколлинеарность)."""
    print("\n--- 3. Проверка на избыточность (высокая корреляция) ---")
    corr_matrix = df[features].corr().abs()
    # Создаем "маску", чтобы убрать дубликаты (верхний треугольник матрицы)
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    redundant_pairs = upper_triangle[upper_triangle > CFG["redundancy_corr_threshold"]].stack().reset_index()
    redundant_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']
    
    return redundant_pairs.to_dict('records')

def check_for_low_variance(df, features):
    """Проверяет признаки на почти нулевую дисперсию."""
    print("\n--- 4. Проверка на низкую/нулевую дисперсию ---")
    variances = df[features].var()
    low_variance_features = variances[variances < CFG["low_variance_threshold"]].to_dict()
    return low_variance_features



In [4]:
# ==============================================================================
# ОСНОВНОЙ СКРИПТ
# ==============================================================================

data_folder = "../data/"
dataset_filename = 'moex_final_dataset.csv'
report_filename = 'reports/feature_report.txt'

# --- ЗАГРУЗКА ДАННЫХ ---
print(f"Загрузка финального датасета из: {data_folder + dataset_filename}")
try:
    df = pd.read_csv(data_folder + dataset_filename)
    df['Date'] = pd.to_datetime(df['Date'])
except FileNotFoundError:
    print("ОШИБКА: Файл не найден. Убедитесь, что все предыдущие скрипты были запущены.")
    exit()

# --- ОПРЕДЕЛЕНИЕ ПРИЗНАКОВ И ЦЕЛЕЙ ---
tbm_cols = [col for col in df.columns if col.startswith('tbm_')]
base_cols_to_exclude = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
features = [col for col in df.columns if col not in base_cols_to_exclude + tbm_cols]

print(f"Найдено {len(features)} признаков для анализа.")
print(f"Найдено {len(tbm_cols)} целевых переменных: {tbm_cols}")

leaky = check_for_data_leakage(df, tbm_cols, features)
non_stationary = check_for_stationarity(df, features)
redundant = check_for_redundancy(df, features)
low_variance = check_for_low_variance(df, features)
multimodal = check_for_multimodality(df, features)



# --- СОСТАВЛЕНИЕ СПИСКА ПОДОЗРИТЕЛЬНЫХ ПРИЗНАКОВ ДЛЯ СТАТИСТИКИ ---
suspicious_features = set(non_stationary.keys()) | set(low_variance.keys()) | set(multimodal.keys())

for item in leaky.values():
    suspicious_features.update(item.keys())
for pair in redundant:
    suspicious_features.add(pair['Feature 1'])
    suspicious_features.add(pair['Feature 2'])

# --- ФОРМИРОВАНИЕ ОТЧЕТА ---
print(f"\n--- Формирование отчета в файл: {report_filename} ---")
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write("="*80 + "\n")
    f.write("ОТЧЕТ ПО ДИАГНОСТИКЕ ПРИЗНАКОВ\n")
    f.write("="*80 + "\n\n")

    # 1. Утечка данных
    f.write("--- 1. ПОДОЗРЕНИЕ НА УТЕЧКУ ДАННЫХ ---\n")
    if leaky:
        for tbm, feats in leaky.items():
            f.write(f"Цель '{tbm}':\n")
            for feat, corr in feats.items():
                f.write(f"  - Признак: {feat}, Корреляция: {corr:.4f}\n")
    else:
        f.write("Признаков с аномально высокой корреляцией к цели не найдено.\n")
    f.write("\n")
    
    # 2. Нестационарность
    f.write("--- 2. ПОДОЗРЕНИЕ НА НЕСТАЦИОНАРНОСТЬ (ADF-тест) ---\n")
    if non_stationary:
        for feat, p_val in sorted(non_stationary.items(), key=lambda item: -item[1]):
            f.write(f"  - Признак: {feat}, Средний p-value: {p_val:.4f} (ВЫШЕ ПОРОГА {CFG['stationarity_pvalue_threshold']})\n")
    else:
        f.write("Нестационарных признаков не найдено.\n")
    f.write("\n")

    # 3. Избыточность
    f.write("--- 3. ПОДОЗРЕНИЕ НА ИЗБЫТОЧНОСТЬ (МУЛЬТИКОЛЛИНЕАРНОСТЬ) ---\n")
    if redundant:
        for pair in redundant:
            f.write(f"  - Пара: {pair['Feature 1']} <--> {pair['Feature 2']}, Корреляция: {pair['Correlation']:.4f}\n")
    else:
        f.write("Сильно коррелирующих пар признаков не найдено.\n")
    f.write("\n")

    # 4. Низкая дисперсия
    f.write("--- 4. ПОДОЗРЕНИЕ НА БЕСПОЛЕЗНОСТЬ (НИЗКАЯ ДИСПЕРСИЯ) ---\n")
    if low_variance:
        for feat, var in low_variance.items():
            f.write(f"  - Признак: {feat}, Дисперсия: {var:.6f}\n")
    else:
        f.write("Признаков с низкой дисперсией не найдено.\n")
    f.write("\n")


    f.write("--- 5. ПОДОЗРЕНИЕ НА МУЛЬТИМОДАЛЬНОСТЬ / ПРОБЛЕМЫ МАСШТАБА ---\n")
    if multimodal:
        for feat, score in sorted(multimodal.items(), key=lambda item: -item[1]):
            f.write(f"  - Признак: {feat}, Оценка разделения кластеров: {score:.2f} (ОЧЕНЬ ВЫСОКАЯ)\n")
    else:
        f.write("Признаков с четко разделенными кластерами значений не найдено.\n")
    f.write("\n")
    # --------------------------------

    # 6. Описательная статистика для "подозреваемых"
    f.write("--- 6. ОПИСАТЕЛЬНАЯ СТАТИСТИКА ДЛЯ ВСЕХ ПОДОЗРИТЕЛЬНЫХ ПРИЗНАКОВ ---\n")
    if suspicious_features:
        stats = df[list(suspicious_features)].describe().transpose()
        f.write(stats.to_string())
    else:
        f.write("Подозрительных признаков для детального анализа не найдено.\n")
        
print("Отчет успешно создан!")

Загрузка финального датасета из: ../data/moex_final_dataset.csv
Найдено 213 признаков для анализа.
Найдено 12 целевых переменных: ['tbm_2d', 'tbm_3d', 'tbm_5d', 'tbm_7d', 'tbm_10d', 'tbm_15d', 'tbm_20d', 'tbm_25d', 'tbm_30d', 'tbm_40d', 'tbm_60d', 'tbm_friday']

--- 1. Проверка на потенциальную утечку данных ---

--- 2. Проверка на нестационарность (ADF-тест) ---


ADF-тест признаков:  76%|███████▌  | 162/213 [00:20<00:06,  7.80it/s]/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
ADF-тест признаков: 100%|██████████| 213/213 [00:27<00:00,  7.85it/s]



--- 3. Проверка на избыточность (высокая корреляция) ---

--- 4. Проверка на низкую/нулевую дисперсию ---

--- 5. Проверка на мультимодальность (Кластеризация) ---


Кластеризация признаков:   0%|          | 0/213 [00:00<?, ?it/s]/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: overflow encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: invalid value encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_p


--- Формирование отчета в файл: reports/feature_report.txt ---
Отчет успешно создан!
